In [ ]:
pip -q install crewai  langchain openai langchain_groq langchain_community crewai_tools langchain_google_genai gradio

In [ ]:
#  Set Groq API key (from Colab secrets)
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("groq_key")

gemini_api_key=userdata.get("gemini_api2")

serper_api_key = userdata.get('serper_api')
os.environ["SERPER_API_KEY"] = serper_api_key

In [ ]:

from crewai import LLM
# llm = LLM(model="gemini/gemini-1.5-flash", api_key=gemini_api_key)
llm = LLM(model="gemini/gemini-2.5-flash", api_key=gemini_api_key)

In [ ]:
from crewai_tools import SerperDevTool ,ScrapeWebsiteTool # Google-style search tool

#Define Search Tool
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:


# === AGENTS ===
search_agent = Agent(
    role="Research Paper Searcher",
    goal="Find recent and relevant academic papers on a topic",
    backstory="You are skilled at finding papers from scholarly sources using search engines.",
    tools=[search_tool],
    llm=llm,
    verbose=True
)

scraper_agent = Agent(
    role="Paper Extractor",
    goal="Fetch the abstract or content from the paper URLs",
    backstory="You're a web scraper that focuses on extracting research content from paper links.",
    tools=[scrape_tool],
    llm=llm,
    verbose=True
)

summarizer_agent = Agent(
    role="Paper Summarizer",
    goal="Summarize the key points from each paper and provide a synthesized summary",
    backstory="You're an NLP expert trained to understand research papers and explain them clearly.",
    llm=llm,
    verbose=True
)

compiler_agent = Agent(
    role="Report Writer",
    goal="Compile a structured report from summaries with titles and URLs",
    backstory="You're a scientific writer focused on creating research briefs with clear structure.",
    llm=llm,
    verbose=True
)

# === TASKS ===
task1 = Task(
    description="Search and return 5 recent academic papers on '{research_topic}'. Return title and URL for each.",
    expected_output="List of 5 papers with title and link.",
    agent=search_agent
)

task2 = Task(
    description="Scrape abstracts or content from each paper URL provided by the search agent.",
    expected_output="List of full abstracts or main content from the 5 paper URLs.",
    agent=scraper_agent
)

task3 = Task(
    description=(
        "Using the content extracted from each paper, perform the following:\n\n"
        "1. For each paper:\n"
        "   - Include the full title\n"
        "   - Include the source URL\n"
        "   - Provide a detailed summary (100 words), including key methods, results, and conclusions\n\n"
        "2. After summarizing all individual papers:\n"
        "   - Write an **overall synthesized summary** that is at least 400-500 words\n"
        "    -write  overall summary of the '{research_topic}' using the information from the extracted papers "
        "   - Capture patterns, contradictions, trends, and overall contributions of the papers as a group\n"
        "   - Reflect critically on the findings (e.g. limitations, gaps, future directions)\n\n"
        "Return a dictionary with two keys:\n"
        "- `overall_summary`: a detailed paragraph combining the papers' insights\n"
        "- `paper_summaries`: a list of dicts with `title`, `url`, and `summary` for each paper"
    ),
    expected_output=(
        "A Python dictionary with:\n"
        "- overall_summary: a single detailed synthesis paragraph (400-500 words)\n"
        "- paper_summaries: list of summaries, each with title, url, summary"
    ),
    agent=summarizer_agent
)

task4 = Task(
    description=(
        "Use the `overall_summary` and `paper_summaries` from the previous task to compile a Markdown report.\n\n"
        "The report should include:\n"
        "- # Research Report: {research_topic}\n"
        "- ## 🧠 Overall Summary\n"
        "  {Insert overall_summary here}\n\n"
        "- ## 📄 Source Papers\n"
        "  For each paper in `paper_summaries`, format as:\n"
        "  ### [Title](URL)\n"
        "  **Summary:** Summary text here\n\n"
        "Generate the full markdown report based on this structure using the actual data from inputs, not template placeholders."
    ),
    expected_output="Complete markdown-formatted research report.",
    agent=compiler_agent
)

# === CREW ===
crew = Crew(
    agents=[search_agent, scraper_agent, summarizer_agent, compiler_agent],
    tasks=[task1, task2, task3, task4],
    verbose=True
)

# === RUN ===
research_topic = input("🔍 Enter your research topic: ")
result = crew.kickoff(inputs={"research_topic": research_topic})







In [ ]:
from IPython.display import Markdown, display

# Clean raw content if it's wrapped in code block
clean_markdown = result.raw.strip("`").replace("markdown", "", 1).strip()

display(Markdown(clean_markdown))

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq


def generate_report(topic: str) -> str:


  # === AGENTS ===
  search_agent = Agent(
      role="Research Paper Searcher",
      goal="Find recent and relevant academic papers on a topic",
      backstory="You are skilled at finding papers from scholarly sources using search engines.",
      tools=[search_tool],
      llm=llm,
      verbose=False
  )

  scraper_agent = Agent(
      role="Paper Extractor",
      goal="Fetch the abstract or content from the paper URLs",
      backstory="You're a web scraper that focuses on extracting research content from paper links.",
      tools=[scrape_tool],
      llm=llm,
      verbose=False
  )

  summarizer_agent = Agent(
      role="Paper Summarizer",
      goal="Summarize the key points from each paper and provide a synthesized summary",
      backstory="You're an NLP expert trained to understand research papers and explain them clearly.",
      llm=llm,
      verbose=False
  )

  compiler_agent = Agent(
      role="Report Writer",
      goal="Compile a structured report from summaries with titles and URLs",
      backstory="You're a scientific writer focused on creating research briefs with clear structure.",
      llm=llm,
      verbose=False
  )

  # === TASKS ===
  task1 = Task(
      description="Search and return 5 recent academic papers on '{research_topic}'. Return title and URL for each.",
      expected_output="List of 5 papers with title and link.",
      agent=search_agent
  )

  task2 = Task(
      description="Scrape abstracts or content from each paper URL provided by the search agent.",
      expected_output="List of full abstracts or main content from the 5 paper URLs.",
      agent=scraper_agent
  )

  task3 = Task(
      description=(
          "Using the content extracted from each paper, perform the following:\n\n"
          "1. For each paper:\n"
          "   - Include the full title\n"
          "   - Include the source URL\n"
          "   - Provide a detailed summary (100 words), including key methods, results, and conclusions\n\n"
          "2. After summarizing all individual papers:\n"
          "   - Write an **overall synthesized summary** that is at least 400-500 words\n"
          "    -write  overall summary of the '{research_topic}' using the information from the extracted papers "
          "   - Capture patterns, contradictions, trends, and overall contributions of the papers as a group\n"
          "   - Reflect critically on the findings (e.g. limitations, gaps, future directions)\n\n"
          "Return a dictionary with two keys:\n"
          "- `overall_summary`: a detailed paragraph combining the papers' insights\n"
          "- `paper_summaries`: a list of dicts with `title`, `url`, and `summary` for each paper"
      ),
      expected_output=(
          "A Python dictionary with:\n"
          "- overall_summary: a single detailed synthesis paragraph (400-500 words)\n"
          "- paper_summaries: list of summaries, each with title, url, summary"
      ),
      agent=summarizer_agent
  )

  task4 = Task(
      description=(
          "Use the `overall_summary` and `paper_summaries` from the previous task to compile a Markdown report.\n\n"
          "The report should include:\n"
          "- # Research Report: {research_topic}\n"
          "- ## 🧠 Overall Summary\n"
          "  {Insert overall_summary here}\n\n"
          "- ## 📄 Source Papers\n"
          "  For each paper in `paper_summaries`, format as:\n"
          "  ### [Title](URL)\n"
          "  **Summary:** Summary text here\n\n"
          "Generate the full markdown report based on this structure using the actual data from inputs, not template placeholders."
      ),
      expected_output="Complete markdown-formatted research report.",
      agent=compiler_agent
  )


  # === CREW ===
  crew = Crew(
      agents=[search_agent, scraper_agent, summarizer_agent, compiler_agent],
      tasks=[task1, task2, task3, task4],
      verbose=False
  )


  result = crew.kickoff(inputs={"research_topic": topic})
  return result.raw.strip("`").replace("markdown", "", 1).strip()


In [ ]:


import gradio as gr

# Wrap the existing report function (update name if needed)
def run_agent_ui(topic):
    return generate_report(topic)

# Gradio UI layout
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 AI Research Assistant")
    gr.Markdown("This tool generates a research report using AI agents. Enter a topic below:")

    with gr.Row():
        topic_input = gr.Textbox(
            label="Research Topic",
            placeholder="e.g., Applications of CRISPR in agriculture",
            lines=1
        )

    with gr.Row():
        generate_button = gr.Button("Generate Report")

    with gr.Row():
        output_box = gr.Textbox(
            label="Generated Report",
            lines=25,
            interactive=False
        )

    generate_button.click(
        fn=run_agent_ui,
        inputs=topic_input,
        outputs=output_box
    )

# Launch the UI
demo.launch()
